### 환경 변수 설정

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

gemini_api_key = os.getenv("GEMINI_API_KEY")

In [3]:

from langchain_core.chat_history import InMemoryChatMessageHistory  # 메모리에 대화 기록을 저장하는 클래스
from langchain_core.runnables.history import RunnableWithMessageHistory  # 메시지 기록을 활용해 실행 가능한 래퍼wrapper 클래스

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", google_api_key=gemini_api_key)

In [4]:
# 세션별 대화 기록을 저장할 딕셔너리
store = {}

# 세션 ID에 따라 대화 기록을 가져오는 함수
def get_session_history(session_id: str):
    # 만약 해당 세션 ID가 store에 없으면, 새로 생성해 추가함
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()  # 메모리에 대화 기록을 저장하는 객체 생성
    return store[session_id]  # 해당 세션의 대화 기록을 반환

# 모델 실행 시 대화 기록을 함께 전달하는 래퍼 객체 생성
with_message_history = RunnableWithMessageHistory(llm, get_session_history)

In [5]:
config = {"configurable": {"session_id": "kim1"}}  # 세션 ID를 설정하는 config 객체 생성

response = with_message_history.invoke(
    [HumanMessage(content="안녕? 난 김일남이야.")],
    config=config,
)

print(response.content)

안녕하세요, 김일남님! 만나서 반갑습니다. 저는 인공지능 어시스턴트입니다.

무엇을 도와드릴까요? 편하게 말씀해주세요.


In [6]:
response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐지?")],
    config=config,
)

print(response.content)

네, 김일남님이세요.


In [7]:
config = {"configurable": {"session_id": "kim2"}}

response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐지?")],
    config=config,
)

response.content

'저는 당신의 이름을 알 수 없습니다.\n\n저는 사용자의 개인 정보를 저장하거나 접근할 수 없는 인공지능이기 때문입니다. 저에게는 별도의 이름이 없으며, 저는 대규모 언어 모델입니다.'

### invoke 메서드 사용
+ 단일 응답 (모든 처리 후 최종 결과 반환)
+ 최종 결과가 한 번에 필요한 경우

In [8]:
config = {"configurable": {"session_id": "kim2"}}

response = with_message_history.invoke(
    [HumanMessage(content="아까 우리가 무슨 얘기 했지?")],
    config=config,
)

response.content

'방금 전 대화는 **당신의 이름에 대한 질문**이었습니다.\n\n제가 당신의 이름을 알 수 없으며, 저는 개인 정보를 저장하지 않는 인공지능이라는 점을 설명해 드렸습니다.\n\n저는 이전 대화 내용을 지속적으로 기억하지 못하고, 각 질문에 대해 새롭게 응답하는 인공지능이기 때문에, 방금 전 질문과 답변 외에는 이전의 대화 내용을 정확히 알기 어렵습니다.'

## 스트림 방식으로 출력하기
+ 스트리밍 응답 (조각 단위로 실시간 반환)
+ 긴 응답을 실시간으로 보여주거나 빠른 피드백이 필요한 경우 유용함
+ for 루프를 통해 각 조각을 순차적으로 처리


In [9]:
config = {"configurable": {"session_id": "kim1"}}
for r in with_message_history.stream(
    [HumanMessage(content = "내가 어느 나라 사람인지 맞춰보고, 그 나라의 문화에 대해 말해봐")],
    config=config,
):
    print(r.content, end="|") # 스트림 방식 출력을 확인하기 위해 end 속성 지정

김일남이라는 성함으로 미루어 짐작컨대, **한국분**이실 가능성이 매우 높습니다.

'김(金)'은 한국에서 가장 흔한| 성씨이며, '일남(一男)' 또한 한국에서 사용되는 이름이기 때문입니다.

그럼 한국 문화에 대해 간략히 소개해 드릴게요.

---

### 한국 문화의 특징

1.  **가족| 및 유교적 가치:**
    *   한국 문화는 전통적으로 유교 사상의 영향을 많이 받아 가족 공동체를 매우 중요하게 여깁니다.
    *   **효(孝)**를 중시하여 부모님과| 어른을 공경하는 것을 중요한 가치로 생각합니다.
    *   나이와 서열을 중시하는 경향이 있으며, 이에 따라 언어(존댓말)와 행동에도 영향을 미칩니다.

2|.  **음식 문화:**
    *   **김치**는 한국인의 밥상에서 빠질 수 없는 대표 음식이며, 발효 식품으로 건강식으로도 알려져 있습니다.
    *   다양한 **반|찬**과 함께 밥을 주식으로 하며, 찌개, 국, 찜, 구이 등 다채로운 조리법이 발달했습니다.
    *   함께 음식을 나누는 **공동체|적인 식사 문화**가 발달해 있습니다.

3.  **한류(韓流)의 확산:**
    *   최근 몇 년간 **K-POP, K-드라마, 한국 영화** 등이| 전 세계적으로 큰 인기를 얻으며 '한류'라는 신조어를 만들어냈습니다.
    *   이러한 대중문화는 한국의 소프트 파워를 상징하며, 한국 문화에 대한 세계인의 관심을| 높이는 데 기여하고 있습니다.

4.  **예절 및 인사:**
    *   상대방에 대한 존중을 표현하는 **인사(고개 숙여 인사하기)**가 일상화되어 있습니다.
|    *   나이나 지위에 따라 **존댓말(높임말)**과 반말을 구분하여 사용하는 언어 예절이 중요합니다.

5.  **전통과 현대의 조화:**
    *   **한|복(韓服)**이라는 아름다운 전통 의상, **한옥(韓屋)**이라는 독특한 건축 양식 등 전통문화를 보존하고 있습니다.
    *   동시에 IT 강국으로 불릴 만큼 **|첨단 